In [12]:
import time
import tensorflow as tf
import numpy as np
import tempfile

from freeman.utils.support_tf import LogLevelManager as llm
llm.set(2)
print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.11.0


## 장치사용

최초 1회는 CPU가 빠르지만, 재실행시에는 GPU가 빠르다.

In [3]:
def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)
    result = time.time() - start
    print(f"10 loops: {1000*result:.2f}ms")
    
print("On CPU:")
with tf.device("CPU:0"):
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)
    
if tf.config.list_physical_devices("GPU"):
    print("On GPU:")
    with tf.device("GPU:0"):
        x = tf.random.uniform([1000, 1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)

On CPU:
10 loops: 43.26ms
On GPU:
10 loops: 2.26ms


In [4]:
tensor_ds = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6])

In [5]:
tensor_ds

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [6]:
for t in tensor_ds:
    print(t)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)


In [15]:
_, filename = tempfile.mkstemp()
with open(filename, "w") as f:
    f.write("""Line 1
Line 2
Line 3
""")
    
file_ds = tf.data.TextLineDataset(filename)
for f in file_ds:
    print(f)

tf.Tensor(b'Line 1', shape=(), dtype=string)
tf.Tensor(b'Line 2', shape=(), dtype=string)
tf.Tensor(b'Line 3', shape=(), dtype=string)


In [16]:
tensor_ds = tensor_ds.map(tf.square).shuffle(3).batch(2)
for t in tensor_ds:
    print(t)
    
file_ds = file_ds.batch(2)
for f in file_ds:
    print(f)

tf.Tensor([ 9 16], shape=(2,), dtype=int32)
tf.Tensor([25  4], shape=(2,), dtype=int32)
tf.Tensor([36  1], shape=(2,), dtype=int32)
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3'], shape=(1,), dtype=string)


In [30]:
def print_ds(msg, ds):
    print(f"{msg}:")
    for x in ds:
        print(x)
        
tensor_ds = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6])
print_ds("Normal", tensor_ds)

tensor_ds = tensor_ds.map(tf.square)
print_ds("Add 1", tensor_ds)

tensor_ds = tensor_ds.shuffle(3)
print_ds("Shuffle", tensor_ds)

tensor_ds = tensor_ds.batch(2)
print_ds("Batch 2", tensor_ds)

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
Normal:
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
Add 1:
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(36, shape=(), dtype=int32)
Shuffle:
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(36, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
Batch 2:
tf.Tensor([9 1], shape=(2,), dtype=int32)
tf.Tensor([25 36], shape=(2,), dtype=int32)
tf.Tensor([ 4 16], shape=(2,), dtype=int32)


In [31]:
layer = tf.keras.layers.Dense(100)
layer

In [32]:
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))
layer

In [34]:
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [35]:
layer = tf.keras.layers.Dense(100)
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 100), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.,

In [36]:
layer.variables

[<tf.Variable 'dense_2/kernel:0' shape=(5, 100) dtype=float32, numpy=
 array([[-0.19438246,  0.00031973, -0.05008858, -0.21432436,  0.15633808,
          0.13022129,  0.01676978,  0.2211606 , -0.13209112, -0.16673535,
         -0.02587058,  0.1080337 , -0.14864221,  0.07312395,  0.11055629,
          0.1571875 , -0.21364637, -0.02930202,  0.09305476, -0.19133049,
          0.2156174 ,  0.1953872 ,  0.04140715, -0.16732273,  0.04075201,
          0.18459348, -0.16076085,  0.21632771, -0.1737741 , -0.08737348,
         -0.07823689,  0.11749403, -0.17338416, -0.06603025, -0.0433792 ,
          0.16107152,  0.21566238,  0.00686321,  0.04763357,  0.1766163 ,
          0.09868561, -0.04542828, -0.2275988 ,  0.09305011,  0.02858846,
         -0.01516245,  0.19299658,  0.02058928,  0.05877419, -0.20509365,
          0.02953978, -0.1891081 , -0.1356293 , -0.20021392, -0.1046851 ,
          0.09329812, -0.11355913, -0.08356926,  0.01188286, -0.02026515,
          0.22182633,  0.07945944, -0.0730

In [37]:
layer.kernel

<tf.Variable 'dense_2/kernel:0' shape=(5, 100) dtype=float32, numpy=
array([[-0.19438246,  0.00031973, -0.05008858, -0.21432436,  0.15633808,
         0.13022129,  0.01676978,  0.2211606 , -0.13209112, -0.16673535,
        -0.02587058,  0.1080337 , -0.14864221,  0.07312395,  0.11055629,
         0.1571875 , -0.21364637, -0.02930202,  0.09305476, -0.19133049,
         0.2156174 ,  0.1953872 ,  0.04140715, -0.16732273,  0.04075201,
         0.18459348, -0.16076085,  0.21632771, -0.1737741 , -0.08737348,
        -0.07823689,  0.11749403, -0.17338416, -0.06603025, -0.0433792 ,
         0.16107152,  0.21566238,  0.00686321,  0.04763357,  0.1766163 ,
         0.09868561, -0.04542828, -0.2275988 ,  0.09305011,  0.02858846,
        -0.01516245,  0.19299658,  0.02058928,  0.05877419, -0.20509365,
         0.02953978, -0.1891081 , -0.1356293 , -0.20021392, -0.1046851 ,
         0.09329812, -0.11355913, -0.08356926,  0.01188286, -0.02026515,
         0.22182633,  0.07945944, -0.07308866,  0.12152

In [38]:
layer.bias

<tf.Variable 'dense_2/bias:0' shape=(100,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

In [41]:
layer = tf.keras.layers.Dense(100)
layer(tf.zeros([10,1]))
layer.variables

[<tf.Variable 'dense_5/kernel:0' shape=(1, 100) dtype=float32, numpy=
 array([[ 0.16497111, -0.19932352,  0.03297424, -0.00488304, -0.1778372 ,
         -0.12457771, -0.19001925,  0.1040251 , -0.06808831, -0.22296475,
         -0.24131815, -0.14313705,  0.21556512, -0.03302412, -0.09812032,
         -0.05331667,  0.07102814,  0.12569106,  0.12841243,  0.0812861 ,
         -0.13574028, -0.11702339, -0.15411851, -0.02014832,  0.10245717,
          0.00764689,  0.09437549, -0.22323246,  0.15967894, -0.17870274,
          0.10041034, -0.04093999,  0.20982495,  0.06837794,  0.00067745,
         -0.21756686,  0.0455173 ,  0.23564824,  0.01767996,  0.00310375,
          0.14486393,  0.04163018, -0.07019262, -0.13061754, -0.19845784,
          0.19575447,  0.2245374 , -0.12329416, -0.00304016, -0.02618787,
         -0.03177375,  0.16010192, -0.17059958, -0.1743648 , -0.20230138,
          0.2042371 , -0.08447985,  0.19218782, -0.09422535,  0.21944174,
          0.19572327,  0.18363687,  0.2016